In [8]:
import pandas as pd
from langdetect import detect
import string
import emoji
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import ENGLISH_STOP_WORDS
from sklearn.metrics.pairwise import cosine_similarity
import mlflow
from mlflow import pyfunc
import requests
import boto3
import json

import nltk
from nltk import word_tokenize
from nltk.stem import WordNetLemmatizer
nltk.download('punkt')
nltk.download('wordnet')
pd.set_option('display.max_rows',500)


[nltk_data] Downloading package punkt to /home/disaster/nltk_data...
[nltk_data]   Package punkt is already up-to-date!
[nltk_data] Downloading package wordnet to /home/disaster/nltk_data...
[nltk_data]   Package wordnet is already up-to-date!


In [17]:
raw_github_data=pd.read_csv('tsv/2020-06-22.tsv',sep='\t',header=0)

In [15]:
raw_github_data_filtered=raw_github_data[(raw_github_data['has_merged_prs']==True)&(raw_github_data['has_readme']==True)&
                                         (pd.isna(raw_github_data['repo_description'])==False)&
                                         (pd.isna(raw_github_data['primary_language_name'])==False)&
                                         (raw_github_data['count_distinct_contributors']>=2)]

In [16]:
def detect_with_error_handle(x):
    try:
        return detect(x)
    except:
        return 'Error'
def has_only_latin_characters(text):
    char_set=string.printable+'-'
    return all(True if x in char_set else False for x in text)
def remove_punctuation(text):
    punctuation_list=string.punctuation+'-'
    return text.translate(str.maketrans('','',punctuation_list))
